# Demo Notebook how to run models on static mouse datasets

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2 

In [2]:
import os
import datajoint as dj
dj.config['database.host'] = os.environ['DJ_HOST']
dj.config['database.user'] = os.environ['DJ_USER']
dj.config['database.password'] = os.environ['DJ_PASS']
dj.config['enable_python_native_blobs'] = True

name = "test"
dj.config['schema_name'] = f"konstantin_nnsysident_{name}"

In [3]:
import torch
import numpy as np
import pickle 

import nnfabrik
from nnfabrik.main import *
from nnfabrik import builder

from nnsysident.datasets.mouse_loaders import static_shared_loaders
from nnsysident.datasets.mouse_loaders import static_loaders
from nnsysident.datasets.mouse_loaders import static_loader

Connecting konstantin@sinzlab.chlkmukhxp6i.eu-central-1.rds.amazonaws.com:3306


# Get Dataloader

In [4]:
# change path here
paths = ['data/static22564-2-12-preproc0.zip']

dataset_fn = 'nnsysident.datasets.mouse_loaders.static_loaders'
dataset_config = dict(
    paths=paths,
    batch_size=64,
)
dataloaders = builder.get_data(dataset_fn, dataset_config)

data/static22564-2-12-preproc0 exists already. Not unpacking data/static22564-2-12-preproc0.zip


In [5]:
dataloaders

OrderedDict([('train',
              OrderedDict([('22564-2-12-0',
                            <torch.utils.data.dataloader.DataLoader at 0x7fc756b40310>)])),
             ('validation',
              OrderedDict([('22564-2-12-0',
                            <torch.utils.data.dataloader.DataLoader at 0x7fc756b40640>)])),
             ('test',
              OrderedDict([('22564-2-12-0',
                            <torch.utils.data.dataloader.DataLoader at 0x7fc756b405b0>)]))])

# Get Model

### The old gaussian readout

In [ ]:
model_fn = 'nnvision.models.se_core_gauss_readout'
model_config = {
   'pad_input': False,
   'stack': -1,
   'layers':4,
   'input_kern': 9,
   'gamma_input': 20,
   'gamma_readout': 0.012,
   'hidden_dilation': 1,
   'hidden_kern': 7,
   'hidden_channels': 64,
    'depth_separable': True,
    
}
model = builder.get_model(model_fn=model_fn, model_config=model_config, dataloaders=dataloaders, seed=1000)

### Spatial Transformer

In [ ]:
model_fn = 'nnvision.models.se_core_spatialXfeature_readout'
model_config = {
   'pad_input': False,
   'stack': -1,
   'layers':4,
   'input_kern': 9,
   'gamma_input': 20,
   'gamma_readout': 0.005,
   'hidden_dilation': 1,
   'hidden_kern': 7,
   'hidden_channels': 64,
    'init_noise': 1e-3,
    'depth_separable': True,
    
}
model = builder.get_model(model_fn=model_fn, model_config=model_config, dataloaders=dataloaders, seed=1000)

### The New gaussian readout: change gauss_type for the different modes

In [6]:
model_fn = 'nnsysident.models.models.se2d_fullgaussian2d'
model_config = {
}
model = builder.get_model(model_fn=model_fn, model_config=model_config, dataloaders=dataloaders, seed=1000)

# Get Trainer

In [7]:
trainer_fn = 'nnsysident.training.trainers.standard_trainer'
trainer_config = dict()
trainer = builder.get_trainer(trainer_fn, trainer_config)

# Run Training

In [8]:
score, output, model_state = trainer(model=model, dataloaders=dataloaders, seed=1000)

Epoch 1: 100%|██████████| 70/70 [00:12<00:00,  5.83it/s]


[001|00/05] ---> 0.07131396234035492


Epoch 2: 100%|██████████| 70/70 [00:05<00:00, 13.16it/s]


[002|00/05] ---> 0.12833432853221893


Epoch 3: 100%|██████████| 70/70 [00:05<00:00, 13.51it/s]


...

[074|03/05] -/-> 0.3588438928127289


Epoch 75: 100%|██████████| 70/70 [00:05<00:00, 13.06it/s]


[075|04/05] -/-> 0.35666757822036743


Epoch 76: 100%|██████████| 70/70 [00:05<00:00, 13.06it/s]


[076|05/05] -/-> 0.3584812879562378
Restoring best model after lr decay! 0.358481 ---> 0.359310
Restoring best model! 0.359310 ---> 0.359310
